<a href="https://colab.research.google.com/github/paulsiddhartha0/travelling_salesman_problem/blob/main/TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.0 MB 4.2 MB/s 
     |████████████████████████████████| 408 kB 67.6 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.9 which is incompatible.
tensorflow-metadata 1.10.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.9 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.9 which is incompatible.
google-cloud-bigquery-storage 1.1.2 requires protobuf<4.0.0dev, but you have protobuf 4.21.9 which is incompatible.
google-api-core 1.31.

In [ ]:
import pandas as pd
df = pd.read_csv('India Cities LatLng.csv')
df = df[df.city!= 'Kolkata']
cities_lat_dict = df[['city', 'lat']].set_index('city')['lat'].to_dict()
cities_lng_dict = df[['city', 'lng']].set_index('city')['lng'].to_dict()

In [ ]:
df.head()

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Delhi,28.6600,77.2300,India,IN,Delhi,admin,29617000.0,16753235.0
1,Mumbai,18.9667,72.8333,India,IN,Mahārāshtra,admin,23355000.0,12478447.0
3,Bangalore,12.9699,77.5980,India,IN,Karnātaka,admin,13707000.0,8443675.0
4,Chennai,13.0825,80.2750,India,IN,Tamil Nādu,admin,11324000.0,6727000.0
5,Hyderabad,17.3667,78.4667,India,IN,Telangana,admin,9746000.0,6993262.0


In [ ]:
from math import cos, asin, sqrt

def DistanceMatrix(x1, y1, x2, y2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((x2 - x1) * p)/2 + cos(x1 * p) * cos(x2 * p) * (1 - cos((y2 - y1) * p)) / 2
    return 12742 * asin(sqrt(a))

In [ ]:
def create_data_model():
  nodes = [0] ## initializing with the depot node as 0
  lat = {}
  lng = {}
  original_id_dict = {}

  depot_node = [0]
  original_id_dict[0] = 'Kolkata'
  lat[0] = 22.5411
  lng[0] = 88.3378


  cust_loc = 1
  for city in cities_lat_dict.keys():
    lat[cust_loc] = cities_lat_dict[city]
    lng[cust_loc] = cities_lng_dict[city]
    original_id_dict[cust_loc] = city
    nodes.append(cust_loc)
    cust_loc +=1

  location = {} 
  distance = {}
  for from_node in nodes:
    location[from_node] = (lat[from_node], lng[from_node])
    distance[from_node] = {}
    for to_node in nodes:
        distance[from_node][to_node] = DistanceMatrix(lat[from_node], lng[from_node],
                                          lat[to_node], lng[to_node])
        
  data = {}
  data["locations"] = location
  data["num_locations"] = len(data["locations"])
  data["num_vehicles"] = 1
  data["depot"] = 0
  data["distance_matrix"] = distance
  data["original_id_dict"] = original_id_dict
  return data





In [ ]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            
            plan_output += ' {} -> '.format(data["original_id_dict"][manager.IndexToNode(index)])
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(data["original_id_dict"][manager.IndexToNode(index)])
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        if route_distance>0:
            print(plan_output)
            total_distance += route_distance
    print('Total Distance of all routes: {}m'.format(total_distance))

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    # Improve the initial solution by a meta-heuristic algorithm

    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    # search_parameters.time_limit.FromSeconds(10)
    search_parameters.time_limit.seconds = 2
    search_parameters.log_search = True

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

In [ ]:
if __name__ == '__main__':
    main()

Objective: 21636
Route for vehicle 0:
 Kolkata ->  Haora ->  Bāli ->  Kāmārhāti ->  Pānihāti ->  Bhātpāra ->  Barddhamān ->  Durgāpur ->  Āsansol ->  Kulti ->  Dhanbad ->  Agartala ->  Āīzawl ->  Imphāl ->  Kohīma ->  Itānagar ->  Shillong ->  Dispur ->  Guwahati ->  Gangtok ->  Shiliguri ->  Purnea ->  Bhāgalpur ->  Begusarai ->  Muzaffarpur ->  Patna ->  Gaya ->  Ranchi ->  Jamshedpur ->  Raurkela ->  Sambalpur ->  Bilāspur ->  Raipur ->  Bhilai ->  Drug ->  Jabalpur ->  Sannai ->  Allahabad ->  Mirzapur ->  Varanasi ->  Gorakhpur ->  Lucknow ->  Cawnpore ->  Etāwah ->  Fīrozābād ->  Agra ->  Bharatpur ->  Mathura ->  Aligarh ->  Shāhjānpur ->  Bareilly ->  Rāmpur ->  Moradabad ->  Sambhal ->  Hāpur ->  Meerut ->  Muzaffarnagar ->  Pānīpat ->  Karnāl ->  Sahāranpur ->  Dehra Dūn ->  Panchkula ->  Chandigarh ->  Shimla ->  Srinagar ->  Handwāra ->  Jammu ->  Amritsar ->  Jalandhar ->  Ludhiāna ->  Patiāla ->  Hisar ->  Rohtak ->  Sonīpat ->  New Delhi ->  Delhi ->  Ghaziabad ->  Farid